In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon , shape
import ast
from shapely.ops import transform
from shapely import wkt
from pyproj import CRS, Transformer

from helper_functions import convert_to_list

In [ ]:
df=pd.read_json("../input/TN/treenation_new.json")
df.head(2)

### Process geometries

Convert nested lists and swap geometries

In [ ]:
df['polygon_data'] = df['polygon_data'].apply(convert_to_list)

Turn into proper geometries

In [ ]:
df['geometry'] = df['polygon_data'].apply(lambda x: Polygon(x) if x is not None and len(x) >= 4 else Polygon())

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry = 'geometry', crs = 'EPSG:4326')
gdf['geometry'] = gdf['geometry'].make_valid()

Explode multipolygons

In [ ]:
gdf = gdf.explode(index_parts=False)

Get area size

In [ ]:
gdf['site_sqkm'] = gdf['geometry'].to_crs(3857).area / 1e6

In [ ]:
gdf['site_sqkm'].describe()

### Preparing columns

In [ ]:
columns_rename_mapping = {
    'project_id': 'project_id_reported',
    'TreeCount': 'trees_planted_reported',
    'description': 'description_reported',
    'created_at': 'planting_date_reported',
    'id': 'site_id_reported',
    'image': 'url',
}

gdf.rename(columns=columns_rename_mapping, inplace=True)

In [ ]:
gdf.drop(columns=["name", "slug", "address", "polygon_data"], inplace=True)

In [ ]:
gdf['host_name'] = 'Tree Nation'

gdf = gdf.assign(species_count_reported=None, species_planted_reported=None,country=None,survival_rate_reported=None)

gdf.info()

### Save it

In [ ]:
gdf.to_file('../input/Plant_for_planet_shapefiles/Tree_Nation.gpkg')